In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps
from scipy.sparse import csr_matrix, find
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from inverted_index import get_inverted_index
from search_engine import search

In [2]:
#nltk.download('stopwords')

# Pré-processando

Definindo algumas constantes

In [3]:
COLUMN_AXIS = 1
CONTENT_COLNAME = 'conteudo'
SUBTITLE_COLNAME = 'subTitulo'
TITLE_COLNAME = 'titulo'
FULL_REPORT_COLNAME = 'noticia'
TOKENS_COLNAME = 'tokens'
TERM_COLNAME = 'term'
REPORT_ID_COLNAME = 'idNoticia'

Carregando dados

In [4]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

Trocando valores nulos por strings vazias nas colunas que representam o conteúdo da notícias. 

In [5]:
empty_str = ""
news.titulo.fillna(empty_str, inplace=True)
news.subTitulo.fillna(empty_str, inplace=True)
news.conteudo.fillna(empty_str, inplace=True)

Concatenando partes da notícia

In [6]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
news[FULL_REPORT_COLNAME] = content

Tokenizando a notícias completa

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

Removendo stopwords

In [8]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

In [9]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

Selecionando apenas as colunas id_noticia e conteudo_noticia do dataframe

In [10]:
news = news[[REPORT_ID_COLNAME, FULL_REPORT_COLNAME]]

Salvando a lista de tokens da notícia em uma nova coluna

In [11]:
news[TOKENS_COLNAME] = news[FULL_REPORT_COLNAME].apply(lambda text: tokenizer.tokenize(text.lower()))

# 1 Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).

In [12]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [13]:
matrix, vocab_to_index = co_occurrence_matrix(tokens)

# 2 Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.

In [14]:
index_to_vocab = {v: k for k, v in vocab_to_index.items()}

In [15]:
three_array_matrix = list(zip(
    matrix.row, matrix.col, matrix.data))

In [16]:
def get_freq(mtuple):
    return mtuple[2]

def get_top3(word, three_array_matrix):
    w_index = vocab_to_index[word]
    freqs = list(filter(lambda mt: mt[0] == w_index, three_array_matrix))
    max_freq = sorted(freqs, key=get_freq, reverse=True)
    sorted_indexes_by_freq = list(map(lambda mtuple: mtuple[1], max_freq))
    top3_index = sorted_indexes_by_freq[:3]
    top3_words = list(map(lambda i: index_to_vocab[i], top3_index))
    return top3_words

# 3 Expanda a consulta original com os termos retornados no passo 2 acima.

In [17]:
def expand_query(term, top3_words):
    expanded_query = " OR ".join([term] + top3_words)
    return expanded_query

# 4 Faça uma busca disjuntiva (OR) considerando a nova consulta.

Para isso importei meus módulos de [InvertedIndex](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/inverted_index.py) e [SearchEngine](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/search_engine.py). Eles já sabem fazer consultas disjuntivas.

In [18]:
inverted_index = get_inverted_index(news)

# 5 Consultando

Escolha livremente três termos de consulta e responda o seguinte:
- Quais os termos retornados para a expansão de cada consulta?  
- Você acha que esses termos são de fato relacionados com a consulta original? Justifique.  
- Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?  
- A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

In [31]:
def print_results(query):
    top3_words = get_top3(query, three_array_matrix)
    print("Termos retornados para expansão:", ", ".join(top3_words))

    query_search_result = search(query, inverted_index)
    query_search_result = sorted(query_search_result)
    query_search_result = list(map(str, query_search_result))
    print("\nDocumentos retornados para consulta:", ", ".join(query_search_result))

    expanded_query = expand_query(query, top3_words)
    expanded_query_search_result = set(search(expanded_query, inverted_index))
    expanded_query_search_result = sorted(list(expanded_query_search_result))
    expanded_query_search_result = list(map(str, expanded_query_search_result))
    print("\nDocumentos retornados para consulta expandida:", ", ".join(expanded_query_search_result))

### Consulta 1: morte

In [32]:
print_results('morte')

Termos retornados para expansão: eduardo, campos, ex

Documentos retornados para consulta: 7, 38, 88, 90, 97, 104, 118, 119, 142, 176, 193, 194, 198, 209, 235, 300, 330, 421, 482, 488, 499, 500, 503, 513, 516, 518, 522, 526, 549, 617, 636, 663, 706, 796, 867, 874, 899, 909, 932, 991, 998, 1002, 1030, 1081, 1089, 1106, 1139, 1166, 1175, 1192, 1202, 1203, 1208, 1235, 1240, 1245, 1254, 1282, 1286, 1290, 1353, 1396, 1397, 1408, 1414, 1432, 1433, 1463, 1467, 1486, 1496, 1500, 1569, 1679, 1690, 1716, 1717, 1800, 1833, 1837, 1843, 1849, 1850, 1862, 1906, 1963, 1997, 2006, 2011, 2020, 2024, 2029, 2034, 2035, 2047, 2052, 2078, 2097, 2142, 2158, 2163, 2171, 2173, 2178, 2179, 2210, 2236, 2246, 2253, 2268, 2284, 2285, 2321, 2331, 2336, 2354, 2374, 2381, 2388, 2392, 2409, 2428, 2431, 2433, 2458, 2462, 2469, 2483, 2484, 2488, 2490, 2498, 2509, 2517, 2521, 2528, 2532, 2552, 2554, 2575, 2577, 2624, 2641, 2678, 2692, 2700, 2716, 2741, 2766, 2769, 2778, 2794, 2806, 2864, 2886, 2890, 2900, 2982, 2987, 30

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**  
- No conjunto de dados parece haver uma grande quantidade de notícias relacionadas à política. E a morte do candidato à presidência Eduardo Campos em período de eleição em 2014 foi um fato bastante noticiado. Portanto faz sentido que à *morte* esteja relacionado seu nome.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**  
- Em ambos os resultados a quantidade de documentos é muito extensa, inviabilizando uma averiguação do conteúdo das notícias.

### Consulta 2: italo

In [22]:
print_results('italo')

Termos retornados para expansão: fittipaldi

Documentos retornados para consulta: 1226

Documentos retornados para consulta expandida: 1165, 1226


**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**  
- O nome *italo* só aparece após o nome *fittipaldi*. *Fittipaldi* é o sobrenome da pessoa cujo nome é Ítalo.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**  
- Na linha de índice 1226, Italo se escreve sem acento. Já no de índice 1165, se escreve com acento. Na busca normal o segundo não é retonado, apesar ser desejado que fosse. Na busca expandida, como Fittipaldi não tem nenhum caractere especial, o segundo documento é capturado pela busca por esse termo. Então, para os casos em que não houve tratamente de caracteres especiais (remoção de acentos, por exemplo), a consulta expandidade parece ter resultados melhores.

### Consulta 3: impeachment

In [54]:
print_results('impeachment')

Termos retornados para expansão: presidente, dilma, fernando

Documentos retornados para consulta: 1, 13, 35, 43, 46, 61, 80, 148, 209, 259, 280, 363, 367, 482, 499, 500, 508, 518, 521, 531, 591, 592, 603, 606, 611, 612, 630, 719, 734, 768, 775, 776, 794, 795, 796, 801, 803, 812, 813, 818, 823, 826, 830, 842, 846, 848, 850, 855, 869, 1030, 1056, 1231, 1379, 2021, 2094, 2268, 2275, 2421, 2509, 3057, 3128, 3400, 3439, 3519, 3552, 3557, 3563, 3575, 3587, 3588, 3650, 4006, 4298, 4619, 4651, 4974, 5561, 5605, 5766, 6037, 6471, 6587, 6627, 6628, 7161, 7167, 7200, 7340, 7344, 7391, 7418, 7420, 7433, 7464, 7995, 8322

Documentos retornados para consulta expandida: 1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 61, 62, 64, 67, 68, 69, 72, 73, 74, 76, 77, 78, 80, 81, 83, 85, 86, 87, 89, 90, 91, 95, 98, 100, 101, 102, 103, 104, 106, 107, 109, 1

**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**  
- Caso semelhante ao da consulta 1. Faz muito sentido que *dilma* e *fernando* sejam termos retornados, pois ambos foram os únicos presidentes na história do Brasil a sofrerem o processo de impeachment. O fato de serem presidentes também justifica o fato de *presidente* ter sido um termo retornado.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**  
- Em ambos os resultados a quantidade de documentos é muito extensa, inviabilizando uma averiguação do conteúdo das notícias.

**A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?** 
- A expansão de consulta aumenta a chance de documentos relevantes serem retornados ao preço de retornar muito mais documentos que não tem muita relevância. Dito isso, a precisão diminui, mas mais documentos relevantes são de fato retornados (portanto o recall aumenta, o que é bom).